In [2]:
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt

import glob
import cv2 

import matplotlib.pylab as plt
from mpl_toolkits import mplot3d
from numpy import linalg
from numpy import random
from scipy.linalg import svd

def create_vector(X,filelists):
    for filelist in filelists[:m]:
        # Open image in an array
        image = cv2.imread(filelist)
        # convert to grayscale (removes 2*l*l dimensions)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Resize image (needed to create the matrix)
        resized_image_gray = cv2.resize(image_gray, (l, l))
        # Reshape the image into a vector
        X_vector = resized_image_gray.reshape(n)
        # Substract the mean
        X_vector = X_vector - np.mean(X_vector)
        #Substract the mean
        X.append(X_vector)
        
def roman_pca(x_matrix,output_dim):
    input_dim = x_matrix.shape[0]
    print('+++, input-dimension', input_dim)
    x_transpose = x_matrix.transpose()
    Sigma = np.matmul(x_transpose,x_matrix)
    #print('+++Sigma:',Sigma)
    eigenValues, eigenVectors = linalg.eig(Sigma)
    idx = eigenValues.argsort()[::-1]
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:, idx]
#    print(eigenValues)
    PCA_matrix = np.empty([input_dim, output_dim])
    PCA_matrix = eigenVectors[:, 0:output_dim]
    return PCA_matrix
def plot_n_components(pca,x_data):
    plt.rcParams["figure.figsize"] = (12,6)
    max_step, n_dim = np.shape(x_data)
    print (max_step, n_dim)
    fig, ax = plt.subplots()
    xi = np.arange(1, int(n_dim)+1, step=1)
    y = np.cumsum(pca.explained_variance_ratio_)
    print (np.shape(xi))
    print (np.shape(y))
    plt.ylim(0.0,1.1)
    plt.plot(xi, y, marker='o', linestyle='--', color='b')

    plt.xlabel('Number of Components')
    plt.xticks(np.arange(0, 51, step=1)) #change from 0-based array index to 1-based human-readable label
    plt.ylabel('Cumulative variance (%)')
    plt.title('The number of components needed to explain variance')

    plt.axhline(y=0.95, color='r', linestyle='-')
    plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

    ax.grid(axis='x')
    plt.show()
    
def svd_clac(X_arr):
    # %% SVD

    # Reshape to get a [n * m] matrix    
    X = np.array(X_arr).T

    # Compute the SVD 
    U,S,V_T = svd(X, full_matrices=False)
    Sigma = np.diag(S)
    return X,U,S,V_T,Sigma

def pca_calc(X,U,pc_n):
    pc = []
    for j in range(X.shape[1]):
        pcj = []
        for i in range(pc_n):
            x = U[:, i] @ X[:, j].T # PC1
            pcj.append(x)
        pc.append(pcj)
    pc = np.array(pc)
    print (np.shape(pc))
    return pc
    

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Get path for files
# Specify the dataset you are trying to extract
file_dir_ls = ["test","train"] #train, test
class_n_ls = ["PNEUMONIA","NORMAL"] # PNEUMONIA, NORMAL
penu_type_ls = ["virus","bacteria"] #bacteria,virus

# create the savetxt directory for the dataset
savetxt_dir = os.getcwd()+"/PCA_dataset/"
if not os.path.isdir(savetxt_dir):
    os.mkdir(savetxt_dir)
    


# %% import database

m = 200 # Number of pictures/file you are extracting; Reduce the size to compute faster
l = 500 # Length of images
n = l*l # Number of pixels
pc_n = 7 # Number of final principal components 

all_x =[]
all_y =[]
for i in range(len(file_dir_ls)):
    file_dir = file_dir_ls[i]
    for j in range(len(class_n_ls)):
        class_n = class_n_ls[j]
        for penu_i in range(len(penu_type_ls)):
            penu_type = penu_type_ls[penu_i]
            path_root = os.getcwd()+'/chest_xray/%s/'%(file_dir)
            print (path_root)
            # Create class 0 normal X matrix
            # The glob function helps you to extract the images from the folder
            if class_n =="PNEUMONIA":
                filelists_norm = glob.glob('%s%s/*%s*.jpeg'%(path_root,class_n,penu_type))#glob.glob(path_root+'/dataset_landscape/*.jpg')
            elif class_n =="NORMAL":
                filelists_norm = glob.glob('%s%s/*.jpeg'%(path_root,class_n))#glob.glob(path_root+'/dataset_landscape/*.jpg')
            # The total number of images that matches the specification 
            original_nb_image = len(filelists_norm)
            print ("original_nb_image:",original_nb_image)

            # Create the original input vector from the images
            X_org = []
            create_vector(X_org,filelists_norm)
            print (np.shape(X_org))
            # Create the labels of the images
            #y_org = []
            if class_n =="PNEUMONIA":
                if penu_type == "bacteria":
                    y_org= np.ones(m, dtype=int)
                    #print(y_org)
                elif penu_type == "virus":
                    y_org= np.ones(m, dtype=int)*2
                    #print(y_org)
            elif class_n =="NORMAL":
                y_org= np.zeros(m, dtype=int)
                #print(y_org)
            if i ==0 and j ==0 and penu_i ==0:
                print ("first round")
                all_x = np.array(X_org)
                all_y = np.array(y_org).reshape(-1,1)
            else:
                all_x = np.r_[all_x,X_org]
                all_y= np.r_[all_y,y_org.reshape(-1,1)]
# combine all the dataset together for PCA
print (np.shape(all_x))
print (np.shape(all_y))

In [ ]:
from sklearn import preprocessing
# normalizing the data before PCA
scaler = preprocessing.MinMaxScaler().fit(all_x)
norm_all_x = scaler.transform(all_x)
     
#y_org = np.array(y_org)
print (all_y)
# svd calculation 
XT, U,S,V_T,Sigma = svd_clac(norm_all_x)
# XT is transpose of X_org
# convert the original input dataset into principal components using the eigenvalues
pca_ls = pca_calc(XT,U,pc_n)

#print (np.shape(pca_ls))

output = np.array(all_y[52:]).reshape(-1,1)
#print (np.shape(output))
pc_out = np.c_[pca_ls,output]
#print (np.shape(pc_out))
np.savetxt("%sPCA%s_%s_%s_all.txt"%(savetxt_dir,pc_n,m,file_dir), pc_out, fmt='%s')

print (np.shape(pc_out))